In [1]:
# imports 
import numpy as np
import pandas as pd
from math import sqrt 
from pprint import pprint
from numpy import array

In [2]:
# load data
import constants
df = pd.read_csv("kddcup.data_10_percent.gz", names = constants.names)

In [3]:
# Categorical features to numeric labels

from sklearn import preprocessing
le_dicts = {}

for categorical_name in constants.categorical_names:
    le = preprocessing.LabelEncoder()
    le.fit(df[categorical_name])
    le_dicts[categorical_name] = dict(zip(le.transform(le.classes_), le.classes_))
    df[categorical_name + '_num'] = le.fit_transform(df[categorical_name])

df['cible'] = df.label.apply(lambda label: 1 if label == 'normal.' else 0)
df['label_four'] = df.label.apply(lambda label: constants.label_to_four_attack_class[label])
df['label_four_num'] = df.label_four.apply(lambda label: constants.five_classes_to_num[label])
df.drop(constants.categorical_names + ['label','label_four_num','label_four'], axis=1, inplace=True)
df.shape

(494021, 42)

In [4]:
# first 5
df.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,protocol_type_num,service_num,flag_num,cible
0,0,181,5450,0,0,0,0,0,1,0,...,0.11,0.0,0.0,0.0,0.0,0.0,1,22,9,1
1,0,239,486,0,0,0,0,0,1,0,...,0.05,0.0,0.0,0.0,0.0,0.0,1,22,9,1
2,0,235,1337,0,0,0,0,0,1,0,...,0.03,0.0,0.0,0.0,0.0,0.0,1,22,9,1
3,0,219,1337,0,0,0,0,0,1,0,...,0.03,0.0,0.0,0.0,0.0,0.0,1,22,9,1
4,0,217,2032,0,0,0,0,0,1,0,...,0.02,0.0,0.0,0.0,0.0,0.0,1,22,9,1


In [5]:
# data to values
df = df.values

In [6]:
#  Deep RandomForest
import argparse
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/kdd.json")
gc = GCForest(config)

In [18]:
# train test 
from sklearn.cross_validation import train_test_split
y = df[:,41]
X = df[:,0:41]

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [8]:
# count of class
len(np.unique(y_train))

2

In [9]:
      # X_enc is the concatenated predict_proba result of RandomForest

X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
   

[ 2018-04-21 16:51:30,207][cascade_classifier.fit_transform] X_groups_train.shape=[(330994, 41)],y_train.shape=(330994,),X_groups_test.shape=[(163027, 41)],y_test.shape=(163027,)
[ 2018-04-21 16:51:30,271][cascade_classifier.fit_transform] group_dims=[41]
[ 2018-04-21 16:51:30,273][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-21 16:51:30,276][cascade_classifier.fit_transform] group_ends=[41]
[ 2018-04-21 16:51:30,278][cascade_classifier.fit_transform] X_train.shape=(330994, 41),X_test.shape=(163027, 41)
[ 2018-04-21 16:51:30,349][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(330994, 41), X_cur_test.shape=(163027, 41)
[ 2018-04-21 16:51:32,742][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.98%
[ 2018-04-21 16:51:35,614][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.97%
[ 2018-04-21 16:51:40,126][kfold_wrapper.log_eval_metrics] Accuracy(layer_0

In [10]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of RandomForest = {:.6f} %".format(acc * 100))

[ 2018-04-21 16:51:58,284][cascade_classifier.transform] X_groups_test.shape=[(163027, 41)]
[ 2018-04-21 16:51:58,308][cascade_classifier.transform] group_dims=[41]
[ 2018-04-21 16:51:58,310][cascade_classifier.transform] X_test.shape=(163027, 41)
[ 2018-04-21 16:51:58,334][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(163027, 41)


Test Accuracy of RandomForest = 99.977918 %


In [11]:
    #  predict_proba result of RandomForest + data
X_test_enc = gc.transform(X_test)
X_train_enc = X_train_enc.reshape((X_train_enc.shape[0], -1))
X_test_enc = X_test_enc.reshape((X_test_enc.shape[0], -1))
X_train_origin = X_train.reshape((X_train.shape[0], -1))
X_test_origin = X_test.reshape((X_test.shape[0], -1))
X_train_enc = np.hstack((X_train_origin, X_train_enc))
X_test_enc = np.hstack((X_test_origin, X_test_enc))

[ 2018-04-21 16:52:01,163][cascade_classifier.transform] X_groups_test.shape=[(163027, 41)]
[ 2018-04-21 16:52:01,186][cascade_classifier.transform] group_dims=[41]
[ 2018-04-21 16:52:01,187][cascade_classifier.transform] X_test.shape=(163027, 41)
[ 2018-04-21 16:52:01,210][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(163027, 41)


In [12]:
# New data
X_train=X_train_enc
X_test=X_test_enc 
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

[ 2018-04-21 16:52:04,087][cascade_classifier.fit_transform] X_groups_train.shape=[(330994, 43)],y_train.shape=(330994,),X_groups_test.shape=[(163027, 43)],y_test.shape=(163027,)
[ 2018-04-21 16:52:04,158][cascade_classifier.fit_transform] group_dims=[43]
[ 2018-04-21 16:52:04,161][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-21 16:52:04,163][cascade_classifier.fit_transform] group_ends=[43]
[ 2018-04-21 16:52:04,165][cascade_classifier.fit_transform] X_train.shape=(330994, 43),X_test.shape=(163027, 43)
[ 2018-04-21 16:52:04,243][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(330994, 43), X_cur_test.shape=(163027, 43)
[ 2018-04-21 16:52:06,583][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.98%
[ 2018-04-21 16:52:08,873][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.98%
[ 2018-04-21 16:52:11,218][kfold_wrapper.log_eval_metrics] Accuracy(layer_0

In [13]:
print("X_train_enc.shape={}, X_test_enc.shape={}".format(X_train_enc.shape, X_test_enc.shape))

X_train_enc.shape=(330994, 2), X_test_enc.shape=(163027, 2)


In [14]:
print("X_train_enc.shape={}, X_test_enc.shape={}".format(X_train.shape, X_test.shape))

X_train_enc.shape=(330994, 43), X_test_enc.shape=(163027, 43)


In [15]:
def update(X_train,X_test):
    # update Estimators
    M = np.delete(X_train, (-1), axis=1)
    X_train= np.delete(M, (-1), axis=1)
    M = np.delete(X_test, (-1), axis=1)
    X_test = np.delete(M, (-1), axis=1)
    return X_train,X_test

def newdata(X_test_enc, X_train_enc):
    # Make Data + Estimators
    X_train_origin = X_train.reshape((X_train.shape[0], -1))
    X_test_origin = X_test.reshape((X_test.shape[0], -1))
    X_train_enc = np.hstack((X_train_origin, X_train_enc))
    X_test_enc = np.hstack((X_test_origin, X_test_enc))
    return X_test_enc, X_train_enc

In [16]:
def affiche():
    # predict
    y_pred = gc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy of RandomForest = {:.6f} %".format(acc * 100),i,'fois')

In [17]:
# deep
for i in range (20):
    X_train,X_test=update(X_train,X_test)
    X_test_enc, X_train_enc=newdata(X_test_enc, X_train_enc)

    X_train=X_train_enc
    X_test=X_test_enc 
    X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    affiche()

[ 2018-04-21 16:52:27,730][cascade_classifier.fit_transform] X_groups_train.shape=[(330994, 43)],y_train.shape=(330994,),X_groups_test.shape=[(163027, 43)],y_test.shape=(163027,)
[ 2018-04-21 16:52:27,797][cascade_classifier.fit_transform] group_dims=[43]
[ 2018-04-21 16:52:27,798][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-21 16:52:27,800][cascade_classifier.fit_transform] group_ends=[43]
[ 2018-04-21 16:52:27,801][cascade_classifier.fit_transform] X_train.shape=(330994, 43),X_test.shape=(163027, 43)
[ 2018-04-21 16:52:27,876][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(330994, 43), X_cur_test.shape=(163027, 43)
[ 2018-04-21 16:52:30,074][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.98%
[ 2018-04-21 16:52:32,272][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.98%
[ 2018-04-21 16:52:34,584][kfold_wrapper.log_eval_metrics] Accuracy(layer_0

('Test Accuracy of RandomForest = 99.984665 %', 0, 'fois')


[ 2018-04-21 16:52:54,014][cascade_classifier.fit_transform] X_groups_train.shape=[(330994, 43)],y_train.shape=(330994,),X_groups_test.shape=[(163027, 43)],y_test.shape=(163027,)
[ 2018-04-21 16:52:54,086][cascade_classifier.fit_transform] group_dims=[43]
[ 2018-04-21 16:52:54,088][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-21 16:52:54,090][cascade_classifier.fit_transform] group_ends=[43]
[ 2018-04-21 16:52:54,091][cascade_classifier.fit_transform] X_train.shape=(330994, 43),X_test.shape=(163027, 43)
[ 2018-04-21 16:52:54,171][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(330994, 43), X_cur_test.shape=(163027, 43)
[ 2018-04-21 16:52:56,685][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.98%
[ 2018-04-21 16:52:59,229][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.98%
[ 2018-04-21 16:53:01,622][kfold_wrapper.log_eval_metrics] Accuracy(layer_0

('Test Accuracy of RandomForest = 99.983438 %', 1, 'fois')


[ 2018-04-21 16:53:20,928][cascade_classifier.fit_transform] X_groups_train.shape=[(330994, 43)],y_train.shape=(330994,),X_groups_test.shape=[(163027, 43)],y_test.shape=(163027,)
[ 2018-04-21 16:53:20,997][cascade_classifier.fit_transform] group_dims=[43]
[ 2018-04-21 16:53:20,999][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-21 16:53:21,000][cascade_classifier.fit_transform] group_ends=[43]
[ 2018-04-21 16:53:21,001][cascade_classifier.fit_transform] X_train.shape=(330994, 43),X_test.shape=(163027, 43)
[ 2018-04-21 16:53:21,076][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(330994, 43), X_cur_test.shape=(163027, 43)
[ 2018-04-21 16:53:23,335][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.98%
[ 2018-04-21 16:53:25,768][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.98%
[ 2018-04-21 16:53:27,925][kfold_wrapper.log_eval_metrics] Accuracy(layer_0

...

('Test Accuracy of RandomForest = 99.985279 %', 17, 'fois')


[ 2018-04-21 17:00:37,275][cascade_classifier.fit_transform] X_groups_train.shape=[(330994, 43)],y_train.shape=(330994,),X_groups_test.shape=[(163027, 43)],y_test.shape=(163027,)
[ 2018-04-21 17:00:37,343][cascade_classifier.fit_transform] group_dims=[43]
[ 2018-04-21 17:00:37,344][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-21 17:00:37,346][cascade_classifier.fit_transform] group_ends=[43]
[ 2018-04-21 17:00:37,347][cascade_classifier.fit_transform] X_train.shape=(330994, 43),X_test.shape=(163027, 43)
[ 2018-04-21 17:00:37,425][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(330994, 43), X_cur_test.shape=(163027, 43)
[ 2018-04-21 17:00:39,842][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.98%
[ 2018-04-21 17:00:42,113][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.98%
[ 2018-04-21 17:00:44,578][kfold_wrapper.log_eval_metrics] Accuracy(layer_0

('Test Accuracy of RandomForest = 99.985279 %', 18, 'fois')


[ 2018-04-21 17:01:04,968][cascade_classifier.fit_transform] X_groups_train.shape=[(330994, 43)],y_train.shape=(330994,),X_groups_test.shape=[(163027, 43)],y_test.shape=(163027,)
[ 2018-04-21 17:01:05,035][cascade_classifier.fit_transform] group_dims=[43]
[ 2018-04-21 17:01:05,037][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-21 17:01:05,039][cascade_classifier.fit_transform] group_ends=[43]
[ 2018-04-21 17:01:05,040][cascade_classifier.fit_transform] X_train.shape=(330994, 43),X_test.shape=(163027, 43)
[ 2018-04-21 17:01:05,117][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(330994, 43), X_cur_test.shape=(163027, 43)
[ 2018-04-21 17:01:07,932][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.98%
[ 2018-04-21 17:01:10,463][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.98%
[ 2018-04-21 17:01:12,903][kfold_wrapper.log_eval_metrics] Accuracy(layer_0

('Test Accuracy of RandomForest = 99.985279 %', 19, 'fois')


# ('Test Accuracy of RandomForest = 99.985892 %', 4, 'fois')